In [8]:
!pip install -U transformers

In [6]:
from huggingface_hub import login
login(new_session=False)

# Fine-tune a small Language Model

why fine tune?

Brcause we want small model for specific task (e.g. don't to pay API credits or leak our data online)

if we have our own model.. we can run it anywhere and everywhere we like.

## How to fine-tune LLM model

we are going to do Supervised Fine-tuning (SFT)

Basically SFT = give samples of inputs and outputs.

For example if our goal was to extract names:

Input: Hello my name is Lokesh
Output: Lokesh

## What we're doing

we're going to build a SLM(small language model) to extract food and drink items from text.

why?

if we needed to go over a large dataset of image captions and filter them for food items (we could them use these filtered captions for a food app).

## Ingredients

1. Model (Gemma3-270M)
2. Dataset (a pre-baked dataset to extract foods from text)

## Methods

1. Download model - Hugging Face `transformers`
2. Download dataset - Hugging Face `datasets`
3. Inspect dataset -  Hugging Face `datasets`
4. Train model on dataset  Hugging Face `trl`
5. Eval model - basically just look at a bunch of samples
6. Create an interactive demo  Hugging Face `gradio`
7. Make the demo public so other people can use it - Hugging Face Spaces

## Fine-tuning LLM vs RAG

* Fine-tuning = To do a very specific task, task, e.g. structured data extraction.
  * An example would be you're an insurance company who gets 10,000 emails a day and you want to extract structured data directly from these emails to JSON.

* RAG = You want to inject custom knowledge into an LLM.
  * An example would be you're an insurance company wanting to send *automatic* responses to people but you want the response to include information from your own docs.

In [1]:
import transformers
import datasets
import torch
import trl # Transformers Reinforcement Learning

# Model

In [2]:
MODEL_NAME = 'google/gemma-3-270m-it'

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype = 'auto',
    device_map = 'auto', # put the model on GPU
    attn_implementation = 'eager' # could use flash_attention_2 but ran into issues.... so stick with Eager

    )


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [5]:
model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)

In [9]:
# Our model requires numbers (tokens) as input.

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    return_tensors="pt"
    )

print(f"[INFO] Model on device: {model.device}")
print(f"[INFO] Model using dtype: {model.dtype}")

[INFO] Model on device: cuda:0
[INFO] Model using dtype: torch.bfloat16


In [23]:
tokens = tokenizer("who are you")
torch.tensor(tokens['input_ids']).unsqueeze(0)

tensor([[    2, 14625,   659,   611]])

In [27]:
outputs = model(torch.tensor(tokens['input_ids']).unsqueeze(0).to(device))
outputs.keys()

odict_keys(['logits', 'past_key_values'])

## Get DataSet

In [32]:
from datasets import load_dataset

# Load the dataset from the hub
dataset = load_dataset("mrdbourke/FoodExtract-1k")

print(f"[INFO] Number of samples in the datasets: {len(dataset['train'])}")
print(f"[INFO] Features in the datasets: {dataset['train'].features}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/616k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1420 [00:00<?, ? examples/s]

[INFO] Number of samples in the datasets: 1420
[INFO] Features in the datasets: {'sequence': Value('string'), 'image_url': Value('string'), 'class_label': Value('string'), 'source': Value('string'), 'char_len': Value('float64'), 'word_count': Value('float64'), 'syn_or_real': Value('string'), 'uuid': Value('string'), 'gpt-oss-120b-label': Value('string'), 'gpt-oss-120b-label-condensed': Value('string'), 'target_food_names_to_use': List(Value('string')), 'caption_detail_level': Value('string'), 'num_foods': Value('float64'), 'target_image_point_of_view': Value('string')}


In [34]:
import random
def get_random_idx(dataset):
  """Returns a random integer index based on number of samples in the datasets"""

  random_idx = random.randint(0, len(dataset)-1)
  return random_idx

random_idx = get_random_idx(dataset['train'])
random_sample = dataset['train'][random_idx]

random_sample

{'sequence': 'Etched Core – Silver Scentsy Tabletop Mini Warmer',
 'image_url': 'https://cdn.shortpixel.ai/client/q_lossy,ret_img,w_600,h_600/https://www.wickfreecandles.co.uk/wp-content/uploads/2020/12/HOME-MiniWarmer-EtchedCoreSilver-ENV-Tabletop-R23-SS21-600x600.jpg',
 'class_label': 'not_food',
 'source': 'coyo700m_dataset',
 'char_len': 49.0,
 'word_count': 8.0,
 'syn_or_real': 'real',
 'uuid': 'ca2fd42f-7210-4792-adf0-46206ebc307f',
 'gpt-oss-120b-label': "{'is_food_or_drink': False, 'tags': [], 'food_items': [], 'drink_items': []}",
 'gpt-oss-120b-label-condensed': 'food_or_drink: 0\ntags: \nfoods: \ndrinks:',
 'target_food_names_to_use': None,
 'caption_detail_level': None,
 'num_foods': None,
 'target_image_point_of_view': None}

In [ ]:
from transformers import pipeline

# load model and use it as a pipeline

pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    device = device,
)

input_text = "Hi my name is Virat"

input_prompt = pipe.tokenizer.apply_chat_template(input_text)
